# 关联分析

In [137]:
# 导入对应的packages
import pandas as pd
import numpy as np
import datetime as dt

## 相邻时间处理函数
- 将相隔10分钟内的告警时间聚类在一起

In [138]:
# 处理相邻时间的函数(相邻为10的时间去除)
def dealData(data_list:list)->list:
    i = 0
    while True:
        if((i+1) >= len(data_list)):
            break
        ymd_i = data_list[i][0:10]
        h_i = int(data_list[i][11:13])
        m_i = int(data_list[i][14:16])
        j = i + 1
        while True:
            if(j >= len(data_list)):
                break
            ymd_j = data_list[j][0:10]
            h_j = int(data_list[j][11:13])
            m_j = int(data_list[j][14:16])
            if(ymd_i==ymd_j):   # 年月日同则可以比较
                if(((h_i==h_j) and (m_j-m_i<=10)) or ((h_j-h_i==1) and (m_j+60-m_i<=10))):   #  同时则比较分钟
                    del(data_list[j])
                else:
                    i = j
                    break
            else:        
                i = j
                break        
    return data_list

## 读取csv文件，然后在读取的同时做简单的数据处理

In [139]:
data = []       # 各测点的经过处理后的数据集
total = []      # 总样本，即经过处理后的全部数据集
for i in range(20):
    file_path = 'res/measure_point_'
    file_path = file_path+str(i)+'.csv'
    df = pd.read_csv(file_path)['alarm_time']     # 读取文件，然后选取alarm_time列，然后选择该列的数据的[0,16)位数据
    for (index,item) in df.items():
        df.loc[index] = item[0:16]                # 处理秒数
    df = list(set(df))
    df.sort()
    df = dealData(df)
    total += df
    data.append(df)                    # 将列表的数据加入到data列表中

In [140]:
data_df = pd.DataFrame(data).T                     # 获取各个报警信号的data_frame
data_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,2021-05-15 10:18,2021-05-15 16:25,2021-05-15 16:34,2021-05-15 16:47,2021-05-15 15:38,2021-05-04 16:04,2021-05-15 10:18,2021-05-04 16:04,2021-05-04 16:04,2021-05-01 19:57,2021-05-15 10:23,2021-05-15 10:24,2021-05-01 06:26,2021-05-04 16:04,2021-05-04 16:04,2021-05-04 16:04,2021-06-18 18:04,2021-05-15 07:18,2021-05-01 04:20,2021-06-22 00:44
1,2021-05-15 16:31,2021-05-15 16:56,None,2021-05-15 17:03,None,2021-05-09 18:25,2021-05-15 16:45,2021-05-15 10:18,2021-05-15 10:18,2021-05-02 18:38,2021-05-15 16:33,None,2021-05-02 06:17,2021-05-09 18:25,2021-05-09 18:25,2021-05-09 18:25,None,2021-05-15 08:00,2021-05-02 05:03,2021-06-24 07:52
2,2021-05-15 17:04,2021-05-18 20:54,None,2021-05-15 17:16,None,2021-05-15 10:18,2021-05-20 09:08,2021-05-20 09:08,2021-05-15 16:31,2021-05-03 18:21,2021-05-15 17:02,None,2021-05-03 05:32,2021-05-15 16:44,2021-05-15 10:18,2021-05-15 10:18,None,2021-05-18 11:15,2021-05-03 05:45,2021-06-25 00:24
3,2021-05-15 17:23,2021-05-18 21:07,None,2021-05-21 13:49,None,2021-05-15 16:06,2021-05-20 11:11,2021-05-20 11:11,2021-05-15 17:04,2021-05-04 15:56,None,None,2021-05-04 05:10,2021-05-15 17:03,2021-05-15 16:06,2021-05-15 16:06,None,2021-06-18 17:55,2021-05-04 06:18,2021-06-25 22:04
4,None,2021-05-20 14:36,None,None,None,2021-05-15 16:31,2021-05-20 14:30,2021-05-20 14:30,2021-05-15 17:23,2021-05-15 10:23,None,None,2021-05-15 10:24,2021-05-15 17:14,2021-05-15 16:31,2021-05-15 16:31,None,2021-06-19 21:10,2021-05-18 11:15,2021-06-27 08:07


## 测点数据与总样本数据对齐

### 对齐函数

In [141]:
def alignmentData(alignment_set:set,total_set:set)->list:
    alignment_set_ok = alignment_set & total_set    # 获取已经对齐的
    rest_set = alignment_set - alignment_set_ok # 获取需要对齐的时间
    for elem1 in rest_set:
        ymd_1 = elem1[0:10]
        h_1 = int(elem1[11:13])
        m_1 = int(elem1[14:16])
        for elem2 in total_set:
            ymd_2 = elem2[0:10]
            h_2 = int(elem2[11:13])
            m_2 = int(elem2[14:16])
            if(ymd_1==ymd_2):
                if((h_1==h_2 and abs(h_1-h_2) <= 10) or (h_1-h_2==1 and m_1+60-m_2<=10) or (h_2-h_1==1 and m_2+60-m_1<=10) ):
                    alignment_set_ok.add(elem2)
                    break
    ret_list = list(alignment_set_ok)
    ret_list.sort()
    return ret_list

### 对齐测点数据

In [147]:
total = dealData(total)         # 将总样本中的相邻10分钟内的数据进行聚类
total = list(set(total))        # 首先使用集合性质去除同元素
total.sort()                    # 总样本排序一下
total_set = set(total)
for i in range(len(data)):
    data[i] = alignmentData(set(data[i]),total_set)
print(total[15:45])
data[0:5]

['2021-05-15 10:18', '2021-05-15 15:38', '2021-05-15 16:06', '2021-05-15 16:25', '2021-05-15 16:44', '2021-05-15 16:56', '2021-05-15 17:14', '2021-05-15 17:28', '2021-05-15 20:35', '2021-05-15 20:48', '2021-05-15 20:59', '2021-05-15 21:27', '2021-05-15 21:43', '2021-05-15 21:59', '2021-05-15 22:11', '2021-05-16 19:49', '2021-05-17 01:01', '2021-05-17 19:16', '2021-05-18 05:23', '2021-05-18 11:15', '2021-05-18 18:01', '2021-05-18 20:54', '2021-05-18 21:07', '2021-05-19 01:02', '2021-05-19 12:54', '2021-05-19 16:33', '2021-05-20 02:09', '2021-05-20 09:08', '2021-05-20 11:11', '2021-05-20 14:30']


[['2021-05-15 10:18', '2021-05-15 16:44', '2021-05-15 17:28'],
 ['2021-05-15 16:25',
  '2021-05-15 16:56',
  '2021-05-18 20:54',
  '2021-05-18 21:07',
  '2021-05-20 14:30'],
 ['2021-05-15 16:44'],
 ['2021-05-15 16:44', '2021-05-15 17:28', '2021-05-21 13:49'],
 ['2021-05-15 15:38']]

## 寻找频繁项集
- 理论概念请参考[频繁项集参考网址](https://blog.csdn.net/u013007900/article/details/54743395?spm=1001.2101.3001.6661.1&utm_medium=distribute.pc_relevant_t0.none-task-blog-2%7Edefault%7ECTRLIST%7Edefault-1-54743395-blog-82707475.pc_relevant_multi_platform_whitelistv2&depth_1-utm_source=distribute.pc_relevant_t0.none-task-blog-2%7Edefault%7ECTRLIST%7Edefault-1-54743395-blog-82707475.pc_relevant_multi_platform_whitelistv2&utm_relevant_index=1)

### 寻找交易集transtion
- 就是找total中的数据在各测点集合出现的测点的集合

In [156]:
def findTranstion(total:list,data:list) ->list:
    ret_list = []
    for elem in total:
        elem_list = []
        for index in range(len(data)):
            if(elem in data[index]):
                elem_list.append(index)
        if(len(elem_list)>1):
            ret_list.append(elem_list)
    return ret_list

In [157]:
transion_list = findTranstion(total,data)
transion_list

[[5, 7, 8, 9, 13, 14, 15],
 [5, 13, 14, 15],
 [0, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15],
 [5, 14, 15],
 [0, 2, 3, 5, 6, 8, 9, 10, 13, 14, 15],
 [0, 3, 8, 9, 10, 12, 13],
 [9, 12],
 [9, 12],
 [5, 13, 14, 15],
 [5, 9, 13, 14, 15],
 [12, 15],
 [17, 18],
 [6, 7, 8],
 [6, 7, 8],
 [1, 6, 7, 8],
 [9, 12],
 [12, 18],
 [12, 18],
 [9, 18],
 [9, 18],
 [9, 18],
 [6, 7, 8, 16, 17],
 [6, 7, 8, 17],
 [6, 7, 8, 17],
 [6, 7, 8, 17],
 [6, 7, 8, 9, 17],
 [9, 18],
 [12, 18]]

### 寻找频繁项集

In [166]:
from apyori import apriori  # 调用 apyori库
L = list(apriori(transactions=transion_list,min_support = 0.25))
L_df = pd.DataFrame(L)
L_df

,items,support,ordered_statistics
0,(5),0.250000,"[((), (5), 0.25, 1.0)]"
1,(6),0.357143,"[((), (6), 0.35714285714285715, 1.0)]"
2,(7),0.357143,"[((), (7), 0.35714285714285715, 1.0)]"
3,(8),0.428571,"[((), (8), 0.42857142857142855, 1.0)]"
4,(9),0.464286,"[((), (9), 0.4642857142857143, 1.0)]"
5,(12),0.321429,"[((), (12), 0.32142857142857145, 1.0)]"
6,(14),0.250000,"[((), (14), 0.25, 1.0)]"
7,(15),0.285714,"[((), (15), 0.2857142857142857, 1.0)]"
8,(18),0.285714,"[((), (18), 0.2857142857142857, 1.0)]"
9,"(5, 14)",0.250000,"[((), (5, 14), 0.25, 1.0), ((5), (14), 1.0, 4...."


### 总结
- 在支持度为0.25情况下，最大频繁项集为{6,7,8}，{5,14,15}